In [2]:
import pandas as pd
import numpy as np
from joblib import load

# Data baru
data_baru = pd.DataFrame({
    'namaproyek': ['Gedung Apartemen', 'Gedung Hotel', 'Gedung Kantor'],
    'waktu': [196, 450, 173],
    'provinsi': ['Jawa Barat', 'DKI Jakarta', 'Kepulauan Riau'],
    'tahun': [2019, 2020, 2021],
    'luas': [17401, 7361, 4324.31],
    'subitem': [4, 6, 4],
    'tinggi': [21.80, 60, 10.50],
    'lantai': [6, 15, 3],
    'ikk': [103.03, 116.84, 116.80],
    'ihbp': [143.72, 103.68, 109.64],
    'inflasi': [3.21, 1.59, 0.86]
})


# Transformasi log pada kolom numerikal
numerical_cols = ['waktu', 'tahun', 'luas', 'subitem', 'tinggi', 'lantai', 'ikk', 'ihbp', 'inflasi']
data_transformed = data_baru.copy()
data_transformed[numerical_cols] = np.log1p(data_transformed[numerical_cols])

# Muat encoder yang telah disimpan
encoder_namaproyek = load('../model/encoder_namaproyek.pkl')
encoder_provinsi = load('../model/encoder_provinsi.pkl')

# Transformasi kolom kategorikal
data_transformed['label_provinsi'] = encoder_provinsi.transform(data_transformed['provinsi'])
data_transformed['label_namaproyek'] = encoder_namaproyek.transform(data_transformed['namaproyek'])

# Muat scaler yang telah disimpan
scaler = load('scaler.pkl')
data_transformed[numerical_cols] = scaler.transform(data_transformed[numerical_cols])

# Hapus kolom yang tidak diperlukan lagi
data_transformed.drop(columns=['lantai', 'namaproyek', 'provinsi'], inplace=True)

# Muat model Voting Regressor yang telah disimpan
voting_regressor2 = load('../model/voting_regressor2_model.joblib')

# Prediksi hasil menggunakan data baru
y_pred_baru = voting_regressor2.predict(data_transformed)

# Invers transformasi log (kembali ke nilai asli)
y_pred_asli = np.expm1(y_pred_baru)

# Buat DataFrame untuk hasil prediksi
hasil_prediksi = data_baru.copy()
hasil_prediksi['Hasil Prediksi'] = y_pred_asli

# Konversi hasil prediksi ke dalam bentuk Rupiah
hasil_prediksi['Hasil Prediksi (Rupiah)'] = hasil_prediksi['Hasil Prediksi'].apply(
    lambda x: f"Rp {x:,.2f}".replace(',', '.').replace('.', ',', 1)
)

# Drop kolom prediksi lama jika tidak diperlukan
hasil_prediksi.drop(columns=['Hasil Prediksi'], inplace=True)

# Tampilkan tabel hasil prediksi
hasil_prediksi

,namaproyek,waktu,provinsi,tahun,luas,subitem,tinggi,lantai,ikk,ihbp,inflasi,Hasil Prediksi (Rupiah)
0,Gedung Apartemen,196,Jawa Barat,2019,17401.00,4,21.8,6,103.03,143.72,3.21,"Rp 25,940.381.192.07"
1,Gedung Hotel,450,DKI Jakarta,2020,7361.00,6,60.0,15,116.84,103.68,1.59,"Rp 59,230.947.181.62"
2,Gedung Kantor,173,Kepulauan Riau,2021,4324.31,4,10.5,3,116.80,109.64,0.86,"Rp 11,262.365.617.68"
